In [2]:
import pandas as pd
import numpy as np

from ast import literal_eval

import statsmodels.formula.api as smf

import setup_modules
from lib.fomc import get_lname

In [3]:
fomc = pd.read_csv('../working-csvs/fomc.csv', index_col=0)

In [4]:
mfdff = pd.read_pickle('../working-csvs/mfdff_selected.pkl')

In [5]:
mfdff

,date,section,lname,svect,voter,sent,region,female,chair,exp,...,svect_tprob_38,svect_tprob_39,svect_tprob_40,svect_tprob_41,svect_tprob_42,svect_tprob_43,svect_tprob_44,final_norm_svect,final_diff_exp_norm,final_diff_prior
19,1994-02-04,2,greenspan,"[-0.29740134824317155, -0.32435034602443663, 0...",1.0,29,governor,0.0,Greenspan,6.490411,...,-1.320819,0.000000,0.0,-1.549192,0.0,-0.500000,1.507937,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
25,1994-02-04,2,mcdonough,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,1,New York,0.0,Greenspan,0.547945,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[0.0, 0.0, 0.0, 3.752776749732569, 2.394498089..."
26,1994-02-04,2,melzer,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,5,St. Louis,0.0,Greenspan,8.684932,...,0.650722,0.000000,0.0,0.000000,0.0,0.000000,0.073532,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[0.0, 0.0, 0.0, 1.8763883748662842, 1.19724904..."
20,1994-02-04,2,jordan,"[0.0, 0.0, 0.3787425226506579, 0.0, 0.0, 0.0, ...",1.0,2,Cleveland,0.0,Greenspan,1.906849,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.621257,"[1.4985287151349502, 0.0, -0.28867513459481275...","[1.7939576251925706, 0.0, -0.46909709371657066...","[1.282303073803447, 0.0, 0.0, 1.25092558324418..."
16,1994-02-04,2,boehne,"[0.0, 0.390893463834136, -1.0, 0.0, 0.0, 0.0, ...",0.0,3,Philadelphia,0.0,Greenspan,13.016438,...,-1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,"[-3.1694598761782635, 0.0, 3.464101615137753, ...","[-2.874030966120643, 0.0, 3.2836796560159947, ...","[-3.7062612859606285, 0.0, 3.7527767497325657,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5885,2018-12-19,2,george,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.41044775...",0.0,5,Kansas City,1.0,Powell,7.221918,...,0.000000,0.000000,0.0,0.000000,0.0,0.589552,0.000000,"[-0.46278877490565806, -0.15395709885664022, 0...","[-0.34867969519931247, -0.43871591198175014, 0...","[-0.48850159507365315, -0.13525398893712612, 0..."
5891,2018-12-19,2,quarles,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0,...",1.0,5,governor,0.0,Powell,1.254795,...,0.000000,0.000000,0.0,1.000000,0.0,-0.457627,0.000000,"[-0.46278877490565806, -0.15395709885664022, 0...","[-0.34867969519931247, -0.43871591198175014, 0...","[-0.4440923591578665, -0.12295817176102375, 0...."
5888,2018-12-19,2,kashkari,"[-0.3987878854106168, 0.0, 0.0, 0.0, -0.520186...",0.0,8,Minneapolis,0.0,Powell,2.967123,...,-0.602902,0.344546,0.0,0.000000,0.0,0.465753,0.000000,"[-0.46278877490565806, -2.2317597358245376, 0....","[-0.34867969519931247, -2.5165185489496475, 0....","[-0.4070846625613776, -2.1905142944155025, 0.6..."
5893,2018-12-19,2,williams,"[0.0, 0.0, 0.42477875017013805, 0.0, 0.7597292...",1.0,26,New York,0.0,Powell,7.805479,...,0.000000,0.562115,0.0,0.000000,0.0,0.818406,1.170169,"[1.5940268979426808, 2.8806621315895997, -0.34...","[1.7081359776490264, 2.5959033184644893, -0.09...","[1.6810452150993749, 3.0904086725690583, -0.34..."


In [6]:
mfdff.columns

Index(['date', 'section', 'lname', 'svect', 'voter', 'sent', 'region',
       'female', 'chair', 'exp',
       ...
       'svect_tprob_38', 'svect_tprob_39', 'svect_tprob_40', 'svect_tprob_41',
       'svect_tprob_42', 'svect_tprob_43', 'svect_tprob_44',
       'final_norm_svect', 'final_diff_exp_norm', 'final_diff_prior'],
      dtype='object', length=123)

In [7]:
mfdff['dist'] = mfdff['final_norm_svect'].map(lambda x : np.sum(np.abs(x)) / len(x))
mfdff['dist_exp'] = mfdff['final_diff_exp_norm'].map(lambda x: np.sum(np.abs(x)) / len(x))
mfdff['dist_prior'] = mfdff['norm_svect_diff_avg_prior'].map(lambda x: np.sum(np.abs(x)) / len(x))

/tmp/ipykernel_13713/3301536847.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['dist'] = mfdff['final_norm_svect'].map(lambda x : np.sum(np.abs(x)) / len(x))
/tmp/ipykernel_13713/3301536847.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['dist_exp'] = mfdff['final_diff_exp_norm'].map(lambda x: np.sum(np.abs(x)) / len(x))
/tmp/ipykernel_13713/3301536847.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

In [8]:
mfdff['dist_prior'].describe()

count    2368.000000
mean        0.492402
std         0.239821
min         0.101544
25%         0.332445
50%         0.440447
75%         0.587707
max         1.824898
Name: dist_prior, dtype: float64

In [9]:
mfdff['date'] = pd.to_datetime(mfdff['date'])
mfdff['merge_date'] = mfdff['date'].dt.strftime('%Y-%m')
mfdff.drop(columns='date', inplace=True)

/tmp/ipykernel_13713/2449098506.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['merge_date'] = mfdff['date'].dt.strftime('%Y-%m')


In [10]:
gb = pd.read_csv('../working-csvs/greenbook.csv', index_col=0)

In [11]:
gb

,merge_date,UNEMPF0,gPPCEF0
0,1967-03,3.8,NaN
1,1967-04,4.0,NaN
2,1967-05,3.9,NaN
3,1967-06,3.9,NaN
4,1967-07,3.9,NaN
...,...,...,...
468,2018-06,3.8,2.0
469,2018-07,3.8,1.4
470,2018-09,3.8,1.5
471,2018-10,3.6,1.9


In [12]:
df = mfdff.merge(gb, how='left', on='merge_date')

In [13]:
panel = df[(df['section'] == 2) & (df['use'] == True)].copy()

In [14]:
panel

,section,lname,svect,voter,sent,region,female,chair,exp,Econ_PhD,...,svect_tprob_44,final_norm_svect,final_diff_exp_norm,final_diff_prior,dist,dist_exp,dist_prior,merge_date,UNEMPF0,gPPCEF0
0,2,greenspan,"[-0.29740134824317155, -0.32435034602443663, 0...",1.0,29,governor,0.0,Greenspan,6.490411,1.0,...,1.507937,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1.362416,1.307361,NaN,1994-02,NaN,NaN
2,2,melzer,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,5,St. Louis,0.0,Greenspan,8.684932,0.0,...,0.073532,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[0.0, 0.0, 0.0, 1.8763883748662842, 1.19724904...",0.458243,0.524663,0.848670,1994-02,NaN,NaN
4,2,boehne,"[0.0, 0.390893463834136, -1.0, 0.0, 0.0, 0.0, ...",0.0,3,Philadelphia,0.0,Greenspan,13.016438,1.0,...,0.000000,"[-3.1694598761782635, 0.0, 3.464101615137753, ...","[-2.874030966120643, 0.0, 3.2836796560159947, ...","[-3.7062612859606285, 0.0, 3.7527767497325657,...",0.640833,0.664564,0.733746,1994-02,NaN,NaN
7,2,lindsey,"[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1.0,4,governor,0.0,Greenspan,2.194521,1.0,...,1.000000,"[0.2162256413315031, 0.0, -0.28867513459481275...","[0.5116545513891234, 0.0, -0.46909709371657066...","[0.3004832062437599, 0.0, -0.5361109642475095,...",0.308551,0.396829,0.404541,1994-02,NaN,NaN
11,2,laware,"[0.0, 0.0, -0.2089830523077163, 0.0, 0.0, 0.0,...",1.0,3,governor,0.0,Greenspan,5.476712,0.0,...,0.295678,"[-0.49132525227171797, 0.0, -0.288675134594812...","[-0.19589634221409763, 0.0, -0.469097093716570...","[-0.5163343078117375, 0.0, -0.3411615227029606...",0.207754,0.280509,0.289812,1994-02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,2,bowman,"[0.30914231879903664, 0.0, 0.0, 0.0, 0.0, 0.0,...",1.0,6,governor,1.0,Powell,0.063014,0.0,...,0.000000,"[-0.46278877490565806, -0.15395709885664022, 0...","[-0.34867969519931247, -0.43871591198175014, 1...","[-0.5427795500818369, -0.15028220993014013, 1....",0.484477,0.580371,0.435672,2018-12,3.7,1.4
2559,2,george,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.41044775...",0.0,5,Kansas City,1.0,Powell,7.221918,0.0,...,0.000000,"[-0.46278877490565806, -0.15395709885664022, 0...","[-0.34867969519931247, -0.43871591198175014, 0...","[-0.48850159507365315, -0.13525398893712612, 0...",0.802300,0.846424,0.548835,2018-12,3.7,1.4
2560,2,quarles,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0,...",1.0,5,governor,0.0,Powell,1.254795,0.0,...,0.000000,"[-0.46278877490565806, -0.15395709885664022, 0...","[-0.34867969519931247, -0.43871591198175014, 0...","[-0.4440923591578665, -0.12295817176102375, 0....",0.472236,0.531045,0.492227,2018-12,3.7,1.4
2561,2,kashkari,"[-0.3987878854106168, 0.0, 0.0, 0.0, -0.520186...",0.0,8,Minneapolis,0.0,Powell,2.967123,0.0,...,0.000000,"[-0.46278877490565806, -2.2317597358245376, 0....","[-0.34867969519931247, -2.5165185489496475, 0....","[-0.4070846625613776, -2.1905142944155025, 0.6...",0.737481,0.809372,0.627775,2018-12,3.7,1.4


In [15]:
panel['gPPCEF0'].notna().value_counts()

gPPCEF0
True     1325
False     439
Name: count, dtype: int64

In [16]:
panel = panel[(panel['gPPCEF0'].notna()) & (panel['UNEMPF0'].notna())].copy()

In [17]:
panel.rename(columns={'gPPCEF0': 'pceinf', 'UNEMPF0': 'unrate'}, inplace=True)

In [21]:
panel['dist'].describe()

count    1325.000000
mean        0.597119
std         0.238876
min         0.000000
25%         0.425660
50%         0.553725
75%         0.708844
max         1.918545
Name: dist, dtype: float64

In [20]:
cov_name_map = {'exp': 'Experience', 'unrate': 'Unemployment',
                'I(exp ** 2)': 'Experience$^2$', 'pceinf': 'PCE inflation', 'C(female)[T.1.0]': 'Female', 'C(voter)[T.1]': 'Voter', 'I(exp ** 2):pceinf': 'Experience$^2$ $\\times$ PCE inflation',
                'I(exp ** 2):unrate': 'Experience$^2$ $\\times$ Unemployment', 'exp:pceinf': 'Experience $\\times$ PCE Inflation', 'exp:unrate': 'Experience $\\times$ Unemployment',
                "C(chair, Treatment(reference='Greenspan'))[T.Yellen]": "Yellen", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]": 'Bernanke', "C(chair, Treatment(reference='Greenspan'))[T.Powell]": "Powell",
                "C(Econ_PhD)[T.1.0]": 'Econ PhD', "C(voter)[T.1.0]": 'Voter'
                }

In [21]:
model1 = smf.ols("dist ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})
model2 = smf.ols("dist ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the committee average in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]","C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('base.tex', 'w') as f:
    f.write(sg.render_latex())
sg

In [22]:
panel['expd']

542      True
548      True
549     False
550      True
552     False
        ...  
2558    False
2559     True
2560    False
2561    False
2562     True
Name: expd, Length: 1325, dtype: bool

In [23]:
inexp_panel = panel[panel['expd'] == False]

In [28]:
inexp_panel['unrate'].describe()

count    519.000000
mean       6.194990
std        1.893204
min        3.700000
25%        4.700000
50%        5.400000
75%        7.800000
max       10.100000
Name: unrate, dtype: float64

In [24]:
model1 = smf.ols("dist_exp ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})
model2 = smf.ols("dist_exp ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the average of members of above first-tertile experience in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance of Experienced Members')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('inexp.tex', 'w') as f:
    f.write(sg.render_latex())
sg

In [29]:
prior_panel = panel[panel['dist_prior'].notna()]

In [32]:
prior_panel['exp'].describe()

count    1212.000000
mean        5.412051
std         4.140975
min         0.002740
25%         2.218493
50%         4.635616
75%         7.687671
max        24.410959
Name: exp, dtype: float64

In [121]:
model1 = smf.ols("dist_prior ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", prior_panel).fit(cov_type='cluster', cov_kwds={'groups': prior_panel['merge_date']})
model2 = smf.ols("dist_prior ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", prior_panel).fit(cov_type='cluster', cov_kwds={'groups': prior_panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the average stance of the members who spoke before them in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Average of Prior Speakers')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]","C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('prior.tex', 'w') as f:
    f.write(sg.render_latex())
sg